In [ ]:
import sys

import autograd
import autograd.numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
sys.path.append("..")

import shared.format
import shared.tools

In [ ]:
import autocrit

In [ ]:
plt.rcParams["font.size"] = shared.format.FONTSIZE

In [ ]:
def f(x):
    return np.power(np.abs(x), 1.5)

grad = autograd.grad(f)
hess = autograd.hessian(f)

def taylor(at_x, from_x, degree=2):
    delta = (at_x - from_x)
    return f(from_x) + grad(from_x) * delta  + 0.5 * delta * hess(from_x) * delta

In [ ]:
btls = autocrit.newtons.NewtonBTLS(f, alpha=1., beta=0.9)

In [ ]:
btls.select_update(-1., 2.)

In [ ]:
alphas = np.linspace(0, 1, num=100)
alpha_acceptable = [
    btls.check_convergence(-1., 2., alpha, 0.2) for alpha in alphas]

In [ ]:
def compute_sufficient_decrease(theta, update_direction, alpha, rho):
    return 2 * rho * alpha * hess(theta) * update_direction * grad(theta)

In [ ]:
acceptable_alphas = alphas[alpha_acceptable]
acceptable_xs = -1 + acceptable_alphas * 2

In [ ]:
btls_alphas = [0.9 ** k for k in range(50)]
btls_xs = [-1. + 2 * btls_alpha for btls_alpha in btls_alphas]

In [ ]:
sufficient_decrease = [compute_sufficient_decrease(-1., 2., alpha, 0.2) for alpha in alphas]
xs_at_alphas = -1 + alphas * 2

In [ ]:
selected_x = np.max(np.array(btls_xs)[btls_xs < np.max(acceptable_xs)])

In [ ]:
fig = plt.figure(figsize=(6, 12))
g = mpl.gridspec.GridSpec(2, 1)

func_ax = fig.add_subplot(g[0, :])
sgn_ax = fig.add_subplot(g[1, :])

xs = np.linspace(-2, 2, num=1000)
func_ax.plot(xs, [f(x) for x in xs], lw=shared.format.LINEWIDTH, color="C1");
func_ax.scatter(-1., f(-1.), color="C1", s=189,
                label=r"$\theta_0$")
func_ax.scatter(selected_x, f(selected_x), color="C1", s=400, marker="*",
                edgecolor="k", linewidth=2, zorder=10,
                label=r"$\theta_{1}$")

func_ax.plot(xs, [taylor(x, -1.) for x in xs],
             lw=shared.format.LINEWIDTH, color="C0", ls="--")
func_ax.scatter(1., taylor(1., -1.),
                color="C0", s=400, marker="*", zorder=10,
                label=r"$\theta_0 + p^\star$")

func_ax.legend(ncol=3)

ymin = -1.
func_ax.hlines(
    ymin, np.min(acceptable_xs), np.max(acceptable_xs),
    lw=shared.format.LINEWIDTH + 4, color="xkcd:brick")
func_ax.set_xlim([-1.5, 1.5])
func_ax.set_ylim([ymin, 4]);
func_ax.set_ylabel(r"$f(\theta)$", fontsize=shared.format.FONTSIZE + 4)

sgn_ymin = 0.
sgn_ax.plot(xs, [np.square(grad(x)) for x in xs],
            lw=shared.format.LINEWIDTH, color="C1")
sgn_ax.scatter(-1., np.square(grad(-1.)), color="C1", s=189)
sgn_ax.scatter(selected_x, np.square(grad(selected_x)), color="C1", s=400, marker="*",
                edgecolor="k", linewidth=2, zorder=10,
                label=r"$\theta_{1}$")
sgn_ax.hlines(
    sgn_ymin, np.min(acceptable_xs), np.max(acceptable_xs),
    lw=shared.format.LINEWIDTH + 4, color="xkcd:brick")
sgn_ax.plot(xs_at_alphas, np.square(grad(-1.)) + sufficient_decrease,
            lw=shared.format.LINEWIDTH, color="grey", ls=":", zorder=0)
sgn_ax.vlines(btls_xs, 0, 0.2)
sgn_ax.set_yticks([0, 1, 2, 3])
sgn_ax.set_xlabel(r"$\theta$", fontsize=shared.format.FONTSIZE + 4)
sgn_ax.set_ylabel(r"$\nabla f(\theta) ^ 2$", fontsize=shared.format.FONTSIZE + 4)
sgn_ax.set_xlim([-1.5, 1.5])
sgn_ax.set_ylim([sgn_ymin, 3.]);
shared.tools.add_panel_label("A", func_ax)
shared.tools.add_panel_label("B", sgn_ax)
plt.savefig("btls.pdf")